Important Notes
- I can also use unsupervised learning in the future to build affinity diagrams. Or perhaps just give each thing a score?
- Include a few "flag" words that the system will automatically flag as one category

In [17]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [9]:
#Attempting git pushing within the notebook
!git add .
!git commit -m "S"
!git push 

[main 7d51549] Slightly improved model by tweaking it!
 1 file changed, 25 insertions(+), 27 deletions(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 832 bytes | 832.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/RudimentaryChef/StudentSurveyNaturalLanguageProcessing.git
   f125f3f..7d51549  main -> main


In [9]:
#Sprint 3 Work (ONGOING)
#Necessary Imports 
import pandas as pd
import random
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from googletrans import Translator




#Working on data augmentation methods within my package
def back_translate(text, from_lang,to_lang):
    try:
        translator = Translator()
        translation = translator.translate(text, src=from_lang, dest=to_lang)
        translation = translator.translate(translation.text, src=to_lang, dest=from_lang)
        return translation.text
    except Exception as e:
        print("ERROR WITH BACK TRANSLATION")
#I can do a while loop of while pd.unique_count max != pd.unique_count min

print(back_translate("I am handicapped","english","filipino"))
#def augment(text):
#Note: I can use a .append() method as required or simply just generate more data using chat gpt

        
#Backtranslates it (times times)

def translate_text(text, lang, times):
    translator = Translator()
    for i in range(times):
        result = translator.translate(text, dest=lang)
        text = result.text
        print(f'Iteration {i+1}: {text}')
    print(translator.translate(text, dest = "english").text)
translate_text("I am disabled and need help for my accomodation. I would appreciate extra time on tests.", "german", 10)

I have a disability
Iteration 1: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 2: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 3: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 4: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 5: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 6: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 7: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 8: Ich bin behindert und brauche Hilfe für meine Unterkunft.Ich würde mich über Tests zusätzliche Zeit freuen.
Iteration 9:

In [2]:
#Updated code for Sprint 2: (Sprint 2 complete)

#Necessary Imports 
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

#Note: this is the link to the real file. The github will have a sample/fake version uploaded to it named "SprintTwoSampleData"
#Change the string to include the location of where you save the xlsx of your "real data file"
response_file = pd.read_excel("/Users/adi/Desktop/RandomCode/Confidential/RealData.xlsx")

#Puts all responses into a list while making sure to fill blank responses
#Basic Data Cleaning

concerns = response_file["concerns"].fillna("NONE").values.tolist()
concernCat = response_file["concerns category"].fillna("NONE").values.tolist()

anything = response_file["anything else"].fillna("NONE").values.tolist()
anythingCat = response_file["anything else category"].fillna("NONE").values.tolist()

pd.unique(response_file["anything else category"])

#response_file["anything else category"].tail(50)
#response_file[response_file['concerns category'] == 'OT'].index

categoryConcerns = response_file['concerns category'].values.tolist()
categoryAnything = response_file["anything else category"].values.tolist()
mapping = {"NC": 0, "LM": 1, "TM": 2, "OT": 3}

#Maps it
categoryConcerns = [mapping[cat] for cat in categoryConcerns]
categoryAnything = [mapping[cat] for cat in categoryAnything]
categoryConcerns
CombinedCats = categoryConcerns + categoryAnything
CombinedResponses = concerns + anything

#Builds a tokenizer and tokenizes the test data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(CombinedResponses)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(CombinedResponses)

#finds the max length. "longest response"
max_sequence_length = max([len(seq) for seq in sequences])

#pads all the sequences to be about the same length. By specifying padding as post we are adding padding to the end.
sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
#Splits data into train and test
X_train, X_test, y_train, y_test = train_test_split(sequences, CombinedCats, test_size=0.2, random_state=42)
model = keras.Sequential()

model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=max_sequence_length))

#Flattens each tensor so its taking only one vector at a time
model.add(Flatten())

#Adds the "classifier layer" This uses at type of activation called Relu. It has 64 neurons to think with. 
model.add(Dense(64,activation = "relu"))
#This is the output layer. It stores the probability of each possible distribution
#Number of possibilities
pd.unique(response_file["anything else category"]).size
poss = max(pd.unique(response_file["anything else category"]).size, pd.unique(response_file["concerns category"]).size)

model.add(Dense(poss, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#training and testing it
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
#The model sees the training set 10 times and each "training set" consists of 2 training datas
model.fit(X_train, y_train, epochs=10, batch_size=2)
#Looks at the loss and accuracy of the model
#The model's loss is how far away from the actual value it is on average. 
#For example if the model says that an IND has a 1% chance being FAQ, 22% chance of it being NON and a 77% chance of it being IND
#Then it has a loss of 23% (22 + 1)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.6f}')
print(f'Test accuracy: {accuracy:.6f}')

Epoch 1/10


2023-11-05 13:30:49.250873: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


500/500 [==============================] - 1s 2ms/step - loss: 0.4388 - accuracy: 0.8330
Epoch 2/10
500/500 [==============================] - 1s 2ms/step - loss: 0.2212 - accuracy: 0.9190
Epoch 3/10
500/500 [==============================] - 1s 3ms/step - loss: 0.0519 - accuracy: 0.9870
Epoch 4/10
500/500 [==============================] - 1s 3ms/step - loss: 0.0092 - accuracy: 0.9990
Epoch 5/10
500/500 [==============================] - 1s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 6/10
500/500 [==============================] - 2s 3ms/step - loss: 6.3570e-04 - accuracy: 1.0000
Epoch 7/10
500/500 [==============================] - 2s 3ms/step - loss: 2.7998e-04 - accuracy: 1.0000
Epoch 8/10
500/500 [==============================] - 2s 4ms/step - loss: 1.8766e-04 - accuracy: 1.0000
Epoch 9/10
500/500 [==============================] - 2s 3ms/step - loss: 1.1460e-04 - accuracy: 1.0000
Epoch 10/10
8/8 [==============================] - 0s 1ms/step - loss: 0.3800 - accuracy: 0.892

In [8]:
#Testing sample responses using our model

sample_response_text = "I need an accomodation for my dyslexia"  # Replace with test
sample_response_sequence = tokenizer.texts_to_sequences([sample_response_text])
sample_response_padded = pad_sequences(sample_response_sequence, maxlen=max_sequence_length, padding='post')
predicted_category = model.predict(sample_response_padded)
#The predicted label of the category
reverse_mapping = { 0: "NC", 1: "LM", 2: "TM", 3: "OT"}
predicted_label = reverse_mapping[np.argmax(predicted_category)]
#Prints the label it is supposed to be
#print(f"Predicted Category: {predicted_label}")
print(predicted_category)

1/1 [==============================] - 0s 18ms/step
[[5.3165553e-05 7.0848070e-02 2.3047657e-01 6.9862217e-01]]


In [80]:
#All code for Sprint 1:

#Necessary Imports 
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
#Loads Files
allFiles = pd.read_csv("TestData.csv")

#Puts all the responses in a list and fills in blank responses with "NONE"
response = allFiles['Responses'].fillna("NONE").values.tolist()
response

category = allFiles['Category'].values.tolist()
#Assigns each label a value. Basically converts the words to a number.
mapping = {"NON": 0, "FAQ": 1, "IND": 2}
category = [mapping[cat] for cat in category]
category

allFiles.head(60)

#Builds a tokenizer and tokenizes the test data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(response)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(response)

category = np.array(category)

#finds the max length. "longest response"
max_sequence_length = max([len(seq) for seq in sequences])

#pads all the sequences to be about the same length. By specifying padding as post we are adding padding to the end.
sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
#Splits data into train and test
X_train, X_test, y_train, y_test = train_test_split(sequences, category, test_size=0.2, random_state=42)
model = keras.Sequential()

#This adds layers to the model. The input dimension is the number of unique "word_index" words (plus 1 for the padding word)
#The output dimension is arbitrary and might need adjustment
#The input length is set to the max sequence length since thats the biggest possible input (after the padding. Expected sequence length) 
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=max_sequence_length))

#Flattens each tensor so its taking one vector at a time from the input layer
model.add(Flatten())
#Adds the "classifier layer" This uses at type of activation called Relu. It has 64 neurons to think with. 
model.add(Dense(64,activation = "relu"))
#This is the output layer. It stores the probability of each possible distribution
model.add(Dense(3, activation='softmax'))

#Compiles the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#The model sees the training set 10 times and each "training set" consists of 2 training datas
model.fit(X_train, y_train, epochs=10, batch_size=2)
#Looks at the loss and accuracy of the model
#The model's loss is how far away from the actual value it is on average. 
#For example if the model says that an IND has a 1% chance being FAQ, 22% chance of it being NON and a 77% chance of it being IND
#Then it has a loss of 23% (22 + 1)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.6f}')
print(f'Test accuracy: {accuracy:.6f}')

#Testing sample responses using our model
sample_response_text = "handicap"  # Replace with test
sample_response_sequence = tokenizer.texts_to_sequences([sample_response_text])
sample_response_padded = pad_sequences(sample_response_sequence, maxlen=max_sequence_length, padding='post')
predicted_category = model.predict(sample_response_padded)
#The predicted label of the category
reverse_mapping = {0: "NON", 1: "FAQ", 2: "IND"}
predicted_label = reverse_mapping[np.argmax(predicted_category)]
#Prints the label it is supposed to be
#print(f"Predicted Category: {predicted_label}")
print(predicted_category)

Epoch 1/10
23/23 [==============================] - 0s 975us/step - loss: 0.9094 - accuracy: 0.5652
Epoch 2/10
23/23 [==============================] - 0s 966us/step - loss: 0.5343 - accuracy: 0.8261
Epoch 3/10
23/23 [==============================] - 0s 857us/step - loss: 0.3190 - accuracy: 0.8913
Epoch 4/10
23/23 [==============================] - 0s 878us/step - loss: 0.1918 - accuracy: 0.9783
Epoch 5/10
23/23 [==============================] - 0s 920us/step - loss: 0.1279 - accuracy: 0.9565
Epoch 6/10
23/23 [==============================] - 0s 948us/step - loss: 0.0550 - accuracy: 1.0000
Epoch 7/10
23/23 [==============================] - 0s 913us/step - loss: 0.0342 - accuracy: 1.0000
Epoch 8/10
23/23 [==============================] - 0s 975us/step - loss: 0.0235 - accuracy: 1.0000
Epoch 9/10
23/23 [==============================] - 0s 928us/step - loss: 0.0140 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 46ms/step - loss: 0.2450 - accuracy: 0.9167
Te

In [1]:
#Create a validation test
#Necessary Imports 
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
#Loads Files
allFiles = pd.read_csv("TestData.csv")

allFiles.head(60)

,name,id,section,section_id,submitted,attempt,786512943: What are some of the things that you enjoy about our course so far?,1,"786512943: At this point in the semester, what concerns you the most about taking this course, if any? If you do not have any, you can write ""none"" or ""NA"". To receive full credit for this activity, you will need to write something below.",1.1,"786512943: Anything you would like me to know about before we start our course regarding your individual needs, or impressions of the course so far? If there is nothing, please write NA or something similar, because to receive full credit for this activity, you will need to write something below.",1.2,"10085897: In case we want to reply to your survey responses in an email, what is your preferred name?",1.3,n correct,n incorrect,score,Category,Responses
0,Liam Smith,1001234.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-18 08-27-56 UTC,2.0,I love the flexibility of being able to study ...,0.0,I'm concerned about the level of difficulty.,0.0,NaN,0.0,I go by Mike,0.0,9.0,0.0,8.0,FAQ,I'm concerned about the level of difficulty.
1,Emma Johnson,1005678.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-15 14-53-21 UTC,1.0,NaN,0.0,NaN,0.0,NaN,0.0,Liz,0.0,9.0,0.0,8.0,NON,NaN
2,Noah Brown,1023456.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-17 02-19-45 UTC,2.0,Online courses give me the freedom to travel a...,0.0,I worry about staying motivated and discipline...,0.0,NaN,0.0,Andy,0.0,9.0,0.0,8.0,FAQ,I worry about staying motivated and discipline...
3,Olivia Taylor,1012345.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-12 09-36-14 UTC,1.0,NaN,0.0,I am also worried that I won't do well in this...,0.0,NaN,0.0,NaN,0.0,9.0,0.0,8.0,IND,I am also worried that I won't do well in this...
4,Sophia Anderson,1045678.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-14 18-04-39 UTC,1.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,9.0,0.0,8.0,NON,Nothing
5,Jackson Williams,1009876.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-13 05-29-17 UTC,1.0,The online discussion forums allow me to engag...,0.0,I hope I can effectively communicate with my p...,0.0,NaN,0.0,My friends call me Sammy,0.0,9.0,0.0,8.0,FAQ,I hope I can effectively communicate with my p...
6,Ava Jones,1034567.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-19 16-58-03 UTC,1.0,Having access to recorded lectures means I can...,0.0,I worry about finding adequate resources and m...,0.0,NaN,0.0,Jess,0.0,9.0,0.0,8.0,FAQ,I worry about finding adequate resources and m...
7,Aiden Martinez,1010987.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-10 11-45-28 UTC,2.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,9.0,0.0,8.0,NON,NaN
8,Lucas Davis,1054321.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-16 03-12-50 UTC,2.0,I appreciate not having to rush to make it to ...,0.0,I hope I won't feel isolated and disconnected ...,0.0,NaN,0.0,My preferred name is Alex,0.0,9.0,0.0,8.0,FAQ,I hope I won't feel isolated and disconnected ...
9,Mia Rodriguez,1098765.0,"202402/MATH/1234/QH8/123456, 202402/MATH/1234/...","864219, 537804",2023-01-11 10-40-07 UTC,2.0,NaN,0.0,NaN,0.0,NaN,0.0,Dancing Dynamo Dan,0.0,9.0,0.0,8.0,NON,NaN


In [2]:
#Puts all the responses in a list and fills in blank responses with "NONE"
response = allFiles['Responses'].fillna("NONE").values.tolist()
response

["I'm concerned about the level of difficulty.",
 'NONE',
 'I worry about staying motivated and disciplined in a self-paced online environment.',
 "I am also worried that I won't do well in this course when it comes to the exams due to my test anxiety.",
 'Nothing',
 'I hope I can effectively communicate with my professor for clarifications and questions.',
 'I worry about finding adequate resources and materials for advanced calculus topics.',
 'NONE',
 "I hope I won't feel isolated and disconnected from my peers and classmates.",
 'NONE',
 "I'm concerned about managing my time efficiently to keep up with assignments and lectures.",
 'NONE',
 'I worry about the possibility of distractions at home affecting my focus.',
 'I hope the online platform is user-friendly and conducive to learning calculus concepts.',
 "I'm concerned about not having face-to-face interactions for collaborative problem-solving.",
 'NONE',
 'none',
 'I worry about the assessment methods and how much they impact 

In [3]:
category = allFiles['Category'].values.tolist()
#Assigns each label a value. Basically converts the words to a number.
mapping = {"NON": 0, "FAQ": 1, "IND": 2}
category = [mapping[cat] for cat in category]
category

[1,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1]

In [4]:
#Builds a tokenizer and tokenizes the test data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(response)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(response)

In [5]:
#Puts all the word in a word_index
word_index

{'i': 1,
 'the': 2,
 'none': 3,
 'to': 4,
 'and': 5,
 'about': 6,
 'for': 7,
 'my': 8,
 'worry': 9,
 'a': 10,
 "i'm": 11,
 'concerned': 12,
 'of': 13,
 'in': 14,
 'do': 15,
 'hope': 16,
 'calculus': 17,
 'have': 18,
 'will': 19,
 'be': 20,
 'online': 21,
 "won't": 22,
 'test': 23,
 'it': 24,
 'exams': 25,
 'anxiety': 26,
 'time': 27,
 'how': 28,
 'work': 29,
 'an': 30,
 'no': 31,
 'need': 32,
 'am': 33,
 'that': 34,
 'course': 35,
 'can': 36,
 'with': 37,
 'advanced': 38,
 'topics': 39,
 'assignments': 40,
 'lectures': 41,
 'at': 42,
 'is': 43,
 'learning': 44,
 'concepts': 45,
 'having': 46,
 'face': 47,
 'grade': 48,
 'person': 49,
 'concerns': 50,
 'virtual': 51,
 'able': 52,
 'paper': 53,
 'there': 54,
 'during': 55,
 'extra': 56,
 'adhd': 57,
 'accomodations': 58,
 'make': 59,
 'level': 60,
 'difficulty': 61,
 'staying': 62,
 'motivated': 63,
 'disciplined': 64,
 'self': 65,
 'paced': 66,
 'environment': 67,
 'also': 68,
 'worried': 69,
 'well': 70,
 'this': 71,
 'when': 72,
 'com

In [6]:
sequences


[[11, 12, 6, 2, 60, 13, 61],
 [3],
 [1, 9, 6, 62, 63, 5, 64, 14, 10, 65, 66, 21, 67],
 [1,
  33,
  68,
  69,
  34,
  1,
  22,
  15,
  70,
  14,
  71,
  35,
  72,
  24,
  73,
  4,
  2,
  25,
  74,
  4,
  8,
  23,
  26],
 [75],
 [1, 16, 1, 36, 76, 77, 37, 8, 78, 7, 79, 5, 80],
 [1, 9, 6, 81, 82, 83, 5, 84, 7, 38, 17, 39],
 [3],
 [1, 16, 1, 22, 85, 86, 5, 87, 88, 8, 89, 5, 90],
 [3],
 [11, 12, 6, 91, 8, 27, 92, 4, 93, 94, 37, 40, 5, 41],
 [3],
 [1, 9, 6, 2, 95, 13, 96, 42, 97, 98, 8, 99],
 [1, 16, 2, 21, 100, 43, 101, 102, 5, 103, 4, 44, 17, 45],
 [11, 12, 6, 104, 46, 47, 4, 47, 105, 7, 106, 107, 108],
 [3],
 [3],
 [1, 9, 6, 2, 109, 110, 5, 28, 111, 112, 113, 8, 48],
 [1,
  16,
  1,
  22,
  114,
  115,
  116,
  117,
  118,
  119,
  5,
  120,
  29,
  34,
  14,
  49,
  121,
  122],
 [3],
 [11, 12, 6, 123, 10, 124, 29, 125, 126, 127, 128, 129, 21, 130],
 [1, 9, 6, 131, 132, 5, 2, 133, 7, 134, 14, 30, 21, 135],
 [3],
 [1, 16, 136, 18, 137, 4, 2, 138, 139, 5, 140, 7, 17, 141],
 [1, 142, 18, 14

In [7]:
category = np.array(category)

In [8]:
#finds the max length. "longest response"
max_sequence_length = max([len(seq) for seq in sequences])

In [9]:
#pads all the sequences to be about the same length. By specifying padding as post we are adding padding to the end.
sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')


In [10]:
#Splits data into train and test
X_train, X_test, y_train, y_test = train_test_split(sequences, category, test_size=0.2, random_state=42)

In [11]:
model = keras.Sequential()

In [12]:
#This adds layers to the model. The input dimension is the number of unique "word_index" words (plus 1 for the padding word)
#The output dimension is arbitrary and might need adjustment
#The input length is set to the max sequence length since thats the biggest possible input (after the padding. Expected sequence length) 
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=128, input_length=max_sequence_length))

In [13]:
#Flattens each tensor so its taking one vector at at ime fro the input layer
model.add(Flatten())
#Adds the "classifier layer" This uses at type of activation called Relu. It has 64 neurons to think with. 
model.add(Dense(64,activation = "relu"))
#This is the output layer. It stores the probability of each possible distribution
model.add(Dense(3, activation='softmax'))

In [14]:
#Compiles the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
#The model sees the training set 10 times and each "training set" consists of 2 training datas
model.fit(X_train, y_train, epochs=10, batch_size=2)
#Looks at the loss and accuracy of the model
#The model's loss is how far away from the actual value it is on average. 
#For example if the model says that an IND has a 1% chance being FAQ, 22% chance of it being NON and a 77% chance of it being IND
#Then it has a loss of 23% (22 + 1)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.6f}')
print(f'Test accuracy: {accuracy:.6f}')

Epoch 1/10
23/23 [==============================] - 0s 1ms/step - loss: 0.9288 - accuracy: 0.6087
Epoch 2/10
23/23 [==============================] - 0s 811us/step - loss: 0.5350 - accuracy: 0.7609
Epoch 3/10
 1/23 [>.............................] - ETA: 0s - loss: 0.5704 - accuracy: 0.5000

2023-10-16 19:32:40.874212: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


23/23 [==============================] - 0s 899us/step - loss: 0.4032 - accuracy: 0.7826
Epoch 4/10
23/23 [==============================] - 0s 848us/step - loss: 0.2432 - accuracy: 0.9348
Epoch 5/10
23/23 [==============================] - 0s 866us/step - loss: 0.1416 - accuracy: 0.9783
Epoch 6/10
23/23 [==============================] - 0s 878us/step - loss: 0.0858 - accuracy: 0.9783
Epoch 7/10
23/23 [==============================] - 0s 862us/step - loss: 0.0638 - accuracy: 1.0000
Epoch 8/10
23/23 [==============================] - 0s 892us/step - loss: 0.0529 - accuracy: 0.9783
Epoch 9/10
23/23 [==============================] - 0s 861us/step - loss: 0.0273 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 121ms/step - loss: 0.2379 - accuracy: 0.9167
Test loss: 0.237872
Test accuracy: 0.916667
